In [1]:
#from __future__ import print_function
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten, Input
from keras.layers import LSTM    # can be replaced by RNN or GRU, nothing has to be changed, only accuracy will change
#from keras.datasets import imdb
from keras.models import Model
from keras.utils import plot_model

Using Theano backend.


In [2]:
import random
lines = []
with open("C:\Users\pooja\Desktop\intern\IMDB-data.txt", "r") as file:
    #data = f.read().split('\n')
    for line in file:   # reads line by line
        line = line.strip() # to remove \n
        lines.append(line)
random.shuffle(lines)

In [3]:
train_data = lines[:6000]
test_data = lines[6000:]

thefile = open('C:\Users\pooja\Desktop\intern\IMDB-test-data.txt', 'w')

for item in test_data:
    thefile.write("%s\n" % item)
lines = train_data

In [4]:
Y = [lines[y].split("\t")[0] for y in range(len(lines))]

X = [lines[y].split("\t")[1] for y in range(len(lines))]

In [5]:
import re
def clean_str(string):

    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()

In [6]:
new_X = [clean_str(s) for s in X]

In [7]:
new_X = [s.split(' ') for s in new_X]

In [8]:
max_len = 0
for itn in range(len(Y)):
    if max_len < len(new_X[itn]) :
        max_len = len(new_X[itn])
print(max_len)

41


In [9]:
seq_len = max_len
padding_word = '<PAD/>'
padded_sentences = []
for i in range(len(new_X)):
    sentence = new_X[i]
    num_padding = seq_len - len(sentence)
    new_sentence = sentence + [padding_word] * num_padding
    padded_sentences.append(new_sentence)

In [10]:
from spacy.en import English
nlp = English()

In [11]:
import numpy as np
def build_input_data_nlp(sentences, labels):
    # uses spacy.en and English()
    
    nb_sentences = len(sentences)          # no. of sentences        
    nb_tokens = len(sentences[0])          # = 50 in this case
    delist = sentences[0]
    word_vec_dim = nlp(delist[0].decode('utf8'))[0].vector.shape[0]       #taken 1st word of 1st sentence
    # .vector is a 1-dimensional numpy array of 32-bit floats. The default English model installs vectors for one million vocabulary entries, using the 300-dimensional vectors
    # shape attribute for numpy arrays returns the dimensions of the array. If Y has n rows and m columns, then Y.shape is (n,m). So Y.shape[0] is n
    #print word_vec_dim
    sentences_matrix = np.zeros((nb_sentences, nb_tokens, word_vec_dim))       # np.zero() Return a new array of given shape and type, filled with zeros
    # print sentences_matrix.shape
    for k in xrange(nb_sentences):
        delist = sentences[k]
        m = len(delist)
        for i in xrange(len(delist)):      # xrange is faster compare to range
            tokens = nlp(delist[i].decode('utf8'))       # token = each word (in loop)
            for j in xrange(len(tokens)):
                sentences_matrix[k, i, :] += tokens[j].vector
    ss = sentences_matrix.shape
    print (ss)
    y = np.array(labels)
    x = sentences_matrix
    return [x,y]

In [12]:
X,Y = build_input_data_nlp(padded_sentences, Y)

(6000L, 41L, 300L)


In [13]:
review = Input(shape = (seq_len , 300))

In [2]:
'''max_features = 20000
maxlen = 80  # cut texts after this number of words (among top max_features most common words)
batch_size = 32'''

In [3]:
'''(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = x_train[:4000]
y_train = y_train[:4000]
x_test = x_test[:1000]
y_test = y_test[:1000]
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')'''

4000 train sequences
1000 test sequences


In [11]:
#x_train[1]

array([ 125,   68,    2, 6853,   15,  349,  165, 4362,   98,    5,    4,
        228,    9,   43,    2, 1157,   15,  299,  120,    5,  120,  174,
         11,  220,  175,  136,   50,    9, 4373,  228, 8255,    5,    2,
        656,  245, 2350,    5,    4, 9837,  131,  152,  491,   18,    2,
         32, 7464, 1212,   14,    9,    6,  371,   78,   22,  625,   64,
       1382,    9,    8,  168,  145,   23,    4, 1690,   15,   16,    4,
       1355,    5,   28,    6,   52,  154,  462,   33,   89,   78,  285,
         16,  145,   95])

In [4]:
'''x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)'''

x_train shape: (4000L, 80L)
x_test shape: (1000L, 80L)


In [14]:
print('Build model...')
#model = Sequential()
#model.add(Embedding(max_features, 200, input_length = 80))
reviews = LSTM(128,dropout=0.5, recurrent_dropout=0.5)(review)

#model.add(Flatten())
reviews = Dense(1, activation='sigmoid')(reviews)                 # many to one classification

Build model...


In [15]:
my_model = Model(inputs = review, outputs = reviews) 

In [16]:
# try using different optimizers and different optimizer configs
my_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

plot_model(my_model, show_shapes = True, to_file='model.png')

In [46]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
        X, Y , test_size=0.25,random_state=42)
#X_train[:5]
y_train[:10]

array(['1', '1', '1', '0', '1', '1', '0', '1', '0', '0'], 
      dtype='|S1')

In [48]:
my_model.fit(x_train, y_train, batch_size=32, epochs=4, validation_data=(x_test, y_test))

Train on 4500 samples, validate on 1500 samples
Epoch 1/4
4500/4500 [==============================] - ETA: 91s - loss: 0.6942 - acc: 0.5000 - ETA: 62s - loss: 0.6926 - acc: 0.5469 - ETA: 51s - loss: 0.6923 - acc: 0.5833 - ETA: 46s - loss: 0.6908 - acc: 0.5781 - ETA: 44s - loss: 0.6913 - acc: 0.5813 - ETA: 41s - loss: 0.6922 - acc: 0.5573 - ETA: 41s - loss: 0.6904 - acc: 0.5759 - ETA: 39s - loss: 0.6919 - acc: 0.5586

In [50]:
score, acc = my_model.evaluate(x_test, y_test, batch_size=32)

1500/1500 [==============================] - ETA: 3s - ETA: 3s - ETA: 3s - ETA: 3s - ETA: 3s - ETA: 3s - ETA: 3s - ETA: 3s - ETA: 2s - ETA: 2s - ETA: 2s - ETA: 2s - ETA: 2s - ETA: 2s - ETA: 2s - ETA: 2s

In [51]:
print('Test score:', score)
print('Test accuracy:', acc)

Test score: 0.13499007981
Test accuracy: 0.968666666667


In [53]:
my_model.save('lstm_model.h5')